In [1]:
using JuMP
# using Interact # used for enabling the slider
# using Gadfly
# using Ipopt
using GLPK

In [2]:
using GLPKMathProgInterface

In [3]:
using Pkg
Pkg.status()

    Status `~/.julia/Project.toml`
  [a076750e] CPLEX v0.4.3
  [9961bab8] Cbc v0.4.4
  [aaaa29a8] Clustering v0.12.2
  [60bf3e95] GLPK v0.9.1
  [3c7084bd] GLPKMathProgInterface v0.4.4
  [c91e804a] Gadfly v1.0.1
  [c601a237] Interact v0.9.0
  [b6b21f68] Ipopt v0.4.5
  [4076af6c] JuMP v0.18.5


In [4]:
predictions = [1, 0, 2, 1, 1, 2, 2, 0, 1] + ones(9);

In [26]:
predictions

9-element Array{Float64,1}:
 2.0
 1.0
 3.0
 2.0
 2.0
 3.0
 3.0
 1.0
 2.0

In [5]:
means = [0.04229166666666667
 0.0417625
 0.024754166666666664 
 0.044345833333333334
 0.02324166666666667
 0.07328333333333333
 0.04852083333333334
 0.0212625
 0.10289583333333334]

9-element Array{Float64,1}:
 0.04229166666666667 
 0.0417625           
 0.024754166666666664
 0.044345833333333334
 0.02324166666666667 
 0.07328333333333333 
 0.04852083333333334 
 0.0212625           
 0.10289583333333334 

In [6]:
means *= 1000

9-element Array{Float64,1}:
  42.29166666666667 
  41.7625           
  24.754166666666663
  44.34583333333333 
  23.24166666666667 
  73.28333333333333 
  48.52083333333334 
  21.2625           
 102.89583333333334 

In [7]:
# # Define some input data about the test system
# # Maximum power output of generators
# const g_max = [1000,1000];
# # Minimum power output of generators
# const g_min = [0,300];
# # Incremental cost of generators 
# const c_g = [50,100];
# # Fixed cost of generators
# const c_g0 = [1000,0]
# # Incremental cost of wind generators
# const c_w = 50;
# # Total demand
# const d = 1500;
# # Wind forecast
# const w_f = 200;

In [77]:
g_max

9-element Array{Float64,1}:
  42.29166666666667 
  41.7625           
  24.754166666666663
  44.34583333333333 
  23.24166666666667 
  73.28333333333333 
  48.52083333333334 
  21.2625           
 102.89583333333334 

In [81]:
# Define some input data about the test system
# Maximum power output of generators
g_max = means;
# Minimum power output of generators
# g_min = zeros(9);
g_min =  [10,10, 10, 10, 20, 10, 20, 20, 60];
# Incremental cost of generators 
c_g = [50,100, 30, 20, 30, 70, 60, 45, 30];
# Fixed cost of generators
# c_g0 = [1000,0, 1300, 1500, 1400, 600, 400, 500, 1000]
c_g0 = zeros(9)
# Incremental cost of wind generators
c_w = 50;
# Total demand
d = 350;
# Wind forecast
w_f = 200;

In [82]:
d1 = 50
d2 = 200
d3 = 100

100

In [83]:
function solve_ed(g_max, g_min, c_g, c_w, d, w_f)
    #Define the economic dispatch (ED) model
    ed=Model(solver=Ipopt.IpoptSolver()) 
    
    # Define decision variables    
    @variable(ed, 0 <= g[i=1:size(g_max)[1]] <= g_max[i]) # power output of generators
    @variable(ed, 0 <= w  <= w_f ) # wind power injection

    # Define the objective function
    @objective(ed,Min,sum{c_g[i] * g[i],i=1:size(g_max)[1]}+ c_w * w)

    # Define the constraint on the maximum and minimum power output of each generator
    for i in 1:2
        @constraint(ed,  g[i] <= g_max[i]) #maximum
        @constraint(ed,  g[i] >= g_min[i]) #minimum
    end

    # Define the constraint on the wind power injection
    @constraint(ed, w <= w_f)

    # Define the power balance constraint
    @constraint(ed, sum{g[i], i=1:size(g_max)[1]} + w == d)

    # Solve statement
    solve(ed)
    
    # return the optimal value of the objective function and its minimizers
    return getValue(g), getValue(w), w_f-getValue(w), getObjectiveValue(ed)
end

solve_ed (generic function with 1 method)

In [85]:
function solve_uc(g_max, g_min, c_g, c_w, d, w_f)
    #Define the unit commitment (UC) model
    uc=Model(solver=GLPKSolverMIP()) 
    
    # Define decision variables    
    @variable(uc, 0 <= g[i=1:9] <= g_max[i]) # power output of generators
    @variable(uc, u[i=1:9], Bin) # Binary status of generators
    @variable(uc, 0 <= w  <= w_f ) # wind power injection

    # Define the objective function
    @objective(uc,Min,sum{c_g[i] * g[i],i=1:9})

    # Define the constraint on the maximum and minimum power output of each generator
    for i in 1:9
        @constraint(uc,  g[i] <= g_max[i] * u[i]) #maximum
        @constraint(uc,  g[i] >= g_min[i] * u[i]) #minimum
    end

    # Define the constraint on the wind power injection
    @constraint(uc, w <= w_f)

    # Define the power balance constraint
    @constraint(uc, sum{g[i], i=1:9} == d)

    # Solve statement
    status = solve(uc)
    
    return status, getValue(g), getValue(w), w_f-getValue(w), getValue(u), getObjectiveValue(uc)
end

# Solve the economic dispatch problem


solve_uc (generic function with 1 method)

In [87]:
status,g_opt,w_opt,ws_opt,u_opt,obj=solve_uc(g_max, g_min, c_g, c_w, d, w_f);

  
println("\n")
println("Dispatch of Generators: ", g_opt[:], " MW")
println("Commitments of Generators: ", u_opt[:])
println("Dispatch of Wind: ", w_opt, " MW")
println("Wind spillage: ", w_f-w_opt, " MW") 
println("\n")
println("Total cost: ", obj, "\$")



Dispatch of Generators: [42.2917, 0.0, 24.7542, 44.3458, 23.2417, 42.6875, 48.5208, 21.2625, 102.896] MW
Commitments of Generators: [1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Dispatch of Wind: 0.0 MW
Wind spillage: 200.0 MW


Total cost: 14384.4375$


In [88]:
function solve_uc_decenter(g_max, g_min, c_g, c_w, d, w_f)
    #Define the unit commitment (UC) model
    uc=Model(solver=GLPKSolverMIP()) 
    
    # Define decision variables    
    @variable(uc, 0 <= g[i=1:9] <= g_max[i]) # power output of generators
    @variable(uc, u[i=1:9], Bin) # Binary status of generators
    @variable(uc, 0 <= w  <= w_f ) # wind power injection

    # Define the objective function
    1, 4,5, 9
    @objective(uc,Min,sum{c_g[i] * g[i],i=1:9})

    # Define the constraint on the maximum and minimum power output of each generator
    for i in 1:9
        @constraint(uc,  g[i] <= g_max[i] * u[i]) #maximum
        @constraint(uc,  g[i] >= g_min[i] * u[i]) #minimum
    end

    # Define the constraint on the wind power injection
    @constraint(uc, w <= w_f)

    # Define the power balance constraint
    @constraint(uc, sum{g[i], i=[2, 8]} == d1)
    @constraint(uc, sum{g[i], i=[1,4,5,9]} == d2)
    @constraint(uc, sum{g[i], i=[3,6,7]} == d3)
    # Solve statement
    status = solve(uc)
    
    return status, getValue(g), getValue(w), w_f-getValue(w), getValue(u), getObjectiveValue(uc)
end

# Solve the economic dispatch problem


solve_uc_decenter (generic function with 1 method)

In [89]:
status,g_opt,w_opt,ws_opt,u_opt,obj=solve_uc_decenter(g_max, g_min, c_g, c_w, d, w_f);

  
println("\n")
println("Dispatch of Generators: ", g_opt[:], " MW")
println("Commitments of Generators: ", u_opt[:])
println("Dispatch of Wind: ", w_opt, " MW")
println("Wind spillage: ", w_f-w_opt, " MW") 
println("\n")
println("Total cost: ", obj, "\$")



Dispatch of Generators: [29.5167, 28.7375, 24.7542, 44.3458, 23.2417, 26.725, 48.5208, 21.2625, 102.896] MW
Commitments of Generators: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Dispatch of Wind: 0.0 MW
Wind spillage: 200.0 MW


Total cost: 15502.0625$
